<center>
    <img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">
    <h1> INF285 - Computación Científica </h1>
    <h2> Tarea 4 - Código Base</h2>
    <h2> [S]cientific [C]omputing [T]eam </a> </h2>
    <h2> Versión: 1.01</h2>
</center>

## Contexto

Un laboratorio de investigación se dedica a realizar investigaciones para empresas en múltiples áreas de estudios, como el análisis de poblaciones entre especies, nuevas formas de calcular integrales, o métodos ingeniosos para resolver problemas en general. Usted, como futuro Ingeniero en Informática de la USM, con conocimientos en Física y Matemática, decide realizar su práctica profesional con esta empresa, por lo que a usted le dan el cargo de analista de datos. Su principal función es la de resolver todo tipo de problemas en el laboratorio relacionados con la obtención y procesamiento de datos.

Durante su estadía en la empresa, usted se da cuenta de que varios departamentos dentro del laboratorio están teniendo problemas para poder avanzar con sus investigaciones, y es por ello que su supervisor de práctica le ha pedido a usted que resuelva esos problemas utilizando todos los conocimientos adquiridos en el curso de Computación Científica. 

## Librerías a utilizar - No modificar

In [8]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

### 1. Pérdida de una resistencia (15 puntos)

Dentro del laboratorio se ha producido un accidente en el cual se ha perdido el valor nominal $R$ de una resistencia eléctrica. Para resolver este problema rápidamente, decide armar un circuito sencillo como el de la figura, para luego realizar con un multímetro varias mediciones de la corriente $i$ y del voltaje $v$ en el circuito, las cuales registra en un archivo _muestras.csv_.

<p align='center'><img src='simple-circuito-electrico.png'></p>


#### 1.1 Proponga un modelo que permita estimar el valor de la resistencia $R$, teniendo solamente los valores de la corriente $i$ y el voltaje $v(i)$, y teniendo en cuenta que el modelo debe tener al menos 2 parámetros a ajustar. Explique explícitamente cómo piensa encontrar el valor de $R$ dentro del modelo, y explique el porqué de su razonamiento.
Respuesta: 



Buscamos plantear un modelo que aproxime de buena forma eficiente la resistencia $R$ que viene dada por la expresión:
$$
    R = \frac{v(i)}{i}
$$
El modelo se buscará a través del método de mínimos cuadrados, es decir, se verá de la siguiente forma:
$$
    y = a + bx
$$
Siendo a y b los parámetros que debemos estimar.
$$
    \begin{bmatrix}
        1 & x_1 \\
        1 & x_2 \\
        \vdots & \vdots \\
        1 & x_n
    \end{bmatrix}
    \begin{bmatrix}
        a \\
        b
    \end{bmatrix}
    =   \begin{bmatrix}
            y_1 \\
            y_2 \\
            \vdots \\
            y_n
        \end{bmatrix}
$$
Esto es igual que resolver un sistema de ecuaciones $Ax = b$, pero esta vez el sistema está sobredeterminado.
Para este tipo de problemas, podemos utilizar la factorización $A = QR$, la cual minimiza el error cuadrático. Siguiendo estos pasos:
$$
    QRx = b 
$$
Resolviendo este sistema de ecuaciones obtendremos los parámetros que necesitamos para estimar el valor de R.


#### 1.2 Implemente la función <em>least_squared</em>, la cual recibe como parámetros un vector de datos de entrada $\bold{x} = (x_1, x_2, ..., x_m)$ y un vector de datos de salida $\bold{y} = (y_1, y_2, ..., y_m)$, y debe retornar los valores para $a$ y $b$ que mejor ajusten al modelo $y = f(x)$ propuesto en el inciso anterior a los datos $(x_i, y_i)$ entregados. (5 puntos)

<div class="alert alert-block alert-danger"><b>Está prohibido usar la función de NumPy np.linalg.lstsq o cualquier otra función predefinida que implemente mínimos cuadrados.</b>
</div>

<div class="alert alert-block alert-info">Sí está permitido usar la función np.linalg.solve para resolver un sistema de ecuaciones. </a>
</div>

Respuesta:


In [24]:
def least_squared(x, y):
    '''
    Inputs
    x: ndarray
       vector of inputs [x1, x2, ..., xm]
    
    y: ndarray
       vector of inputs [y1, y2, ..., ym]

    ---
    Returns
    s: ndarray
       vector of outputs [b, a]
    '''
    # Tu codigo aqui
    A = np.array(np.transpose([np.ones(x.size), x]))
    
    Q, R = np.linalg.qr(A, mode='reduced')
    
    s = np.linalg.solve(R, np.dot(Q.T, y))
    return s

#### 1.3 Utilizando los valores del dataset <em>muestras.csv</em> y la función $\verb|least_squared|$, entregue una aproximación de la resistencia $R$ con dos cifras significativas. (5 puntos)

In [26]:
#Realizar aqui sus experimentos
df = pd.read_csv('muestras.csv')
R = np.round(least_squared(df.ampere, df.voltage), 2)
R

array([0.98, 2.46])

### 2. Aproximando un área bajo una curva (25 puntos)

En una de las áreas del laboratorio, se está intentando calcular el área de un terreno irregular para comprobar si es posible construir unos galpones destinados a la mantención de camiones dedicados al transporte de combustible. La región viene representada por:

$$
S = \{(x, y) \in \mathbb{R}^2 : x^4 \leq y \leq e^{\sin(x)}\}
$$

Para resolver este problema, se debe calcular la siguiente integral:

$$
\displaystyle\int_{x_1}^{x_2} (e^{\sin(x)}-x^4) dx
$$

donde $x_1$ y $x_2$ son, respectivamente, el mínimo y el máximo valor posible para $x$ dentro de la región $S$.

Sin embargo, se dieron cuenta de dos detalles importantes:
- Las constantes $x_1$ y $x_2$ no pueden encontrarse de manera analítica.
- La integral no se puede calcular de manera analítica.

Para ello, el propio laboratorio propone calcular cualquier integral mediante la siguiente fórmula de cuadratura:

$$
\displaystyle\int_{x_1}^{x_2} f(x) dx \approx af(x_1) + bf\left(\dfrac{x_1+x_2}{2}\right) + cf(x_2)
$$

donde $a,b,c\in\mathbb{R}$. 

El único problema que se tiene es encontrar los pesos $a,b,c$ de tal forma que la integral resulte exacta para polinomios del grado más alto posible.

Por lo tanto, lo que debe buscar son los valores de los límites de integración $x_1$ y $x_2$, y los pesos $a,b,c$ para la fórmula de cuadratura. 

#### 2.1 Explique teóricamente cómo encontrar los límites de integración $x_1$ y $x_2$, que representan el mínimo y máximo valor posible para $x$ en la región $S$. Justifique su elección analizando la convergencia del algoritmo. <em>Hint: It is not necessary to analyze the integral, just propose an algorithm that finds both constants using the knowledge of the subject</em> (5 puntos)

Respuesta:



Para encontrar los límites de integración $x_1$ y $x_2$ podemos utilizar cualquier método de búsqueda de raíces. En este caso, teniendo en cuenta la fórmula de cuadratura propuesta por el laboratorio, utilizaremos el método Newton, para este caso tomaremos la función $f(x) = (e^{\sin(x)}-x^4)$.
Notar que para este método necesitamos $f'(x)$.


#### 2.2 Una vez encontrados los límites de integración $x_1$ y $x_2$, explique de manera teórica cómo encontrar los valores de los pesos $a$, $b$ y $c$ para la cuadratura propuesta por el laboratorio. <em>Hint: Remember that any quadrature formula must exactly integrate all the polynomials $x^n, \forall n = \{0,1,2,\ldots,k\}$ in $[-1,1]$</em>. (10 puntos)

Respuesta:



Para encontrar los pesos adecuados haremos uso de el método de cuadratura Gaussiana en su versión reducida, por lo que haremos uso del polinomio de Legendre $p_n(x)$ para $n = 2$ con los nodos obtenidos en el punto anterior.
$$
    p_n(x) = \frac{1}{2^n n!}\frac{d^n}{dx^n}[(x^2 -1)^n]
$$ 
$$
    p_0(x) = 1 \\
    p_1(x) = x \\
    p_2(x) = \frac{1}{2}(3x^2 - 1) 
$$
Lo que nos dará los pesos para la cuadratura.


#### 2.3 Implemente las siguientes dos funciones: (5 puntos)
- $\verb|find_integration_limits|$, que calcula los límites de integración $x_1$ y $x_2$.
- $\verb|find_quadrature_weights|$, que calcula los pesos $a,b$ y $c$ de la fórmula de cuadratura.

In [ ]:
def find_integration_limits(g1, g2, x10, x20, tol=1e-5):
    '''
    Inputs:
    g1:  lambda
         function that calculates g1(x)
    g2:  lambda
         function that calculates g2(x)
    x10: int
         Initial coefficient x_1^0
    x20: int
         Initial coefficient x_2^0
    tol: double
         Coefficient of tolerance, defect value = 10^-5
    Outputs:
    x1:  double
         Coefficient x_1
    x2:  double
         Coefficient x_2
    '''
    #Tu codigo aqui
    x1 = x10
    x2 = x20
    f = lambda x: g1(x) - g2(x)
    df = lambda x: (np.exp(np.sin(x)) * np.cos(x)) - (np.power(x, 3) * 4)

    while abs(f(x1)) > tol:
        x1 = x1 - (f(x1) / df(x1))     
    
    while abs(f(x2)) > tol:
        x2 = x2 - (f(x2) / df(x2))
        
    return x1, x2

def find_quadrature_weights(x1, x2):
    '''
    Inputs:
    x1: double
        Coefficient x_1
    x2: double
        Coefficient x_2
    Output:
    a:  double
        Coefficient a
    b:  double
        Coefficient b
    a:  double
        Coefficient c
    '''
    # Tu codigo aqui
    p0 = 0
    p1 = lambda x : x
    p2 = lambda x: (1 / 2) * (3*(x**2) + 1)
    a = p0 + p1(x1) + p2(x1)
    b = p0 + p1(x1) - p1(x2) + p2(x1) - p2(x2)
    
    return a, b, c

#### 2.4 Implemente la función $\verb|quadrature|$ que implementa el lado derecho de la fórmula de cuadratura, y entregue una aproximación del valor del área de la región $S$ utilizando los valores de los límites de integración $x_1,x_2$ y los pesos $a$, $b$ y $c$ de la fórmula de cuadratura. (5 puntos)

In [28]:
def quadrature(f, a, b, c, x1, x2):
    '''
    Inputs:
    f:  lambda
        function that calculates e^(sin(x))-x^4
    a:  double
        Coefficient a
    b:  doube
        Coefficient b
    c:  double
        Coefficient c
    x1: double
        Coefficient x_1
    x2: double
        Coefficient x_2
    Output:
    I:  double
        Returned value used by the quadrature formula
    '''
    #Tu codigo aqui
    f = np.vectorize(f)
    x = np.linspace()
    
    return I

# No modificar
x10 = -1
x20 = 1

#Tus experimentos aqui
g1 = lambda x: np.exp(np.sin(x))
g2 = lambda x: np.power(x, 4)
x1, x2 = find_integration_limits(g1, g2, x10, x20)
x1, x2


KeyboardInterrupt: 

### 3. Desacuerdos en las ecuaciones. (50 puntos)

Dentro del laboratorio, una de las áreas de estudio es la concentración de poblaciones de 2 especies distintas, denominadas $x_1$ y $x_2$ respectivamente. En este estudio estuvieron involucrados 2 científicos, los cuales propusieron 6 ecuaciones diferenciales usando enfoques distintos. Lamentablemente, los científicos no lograron un consenso en la formulación del modelo. Por ende, le piden a usted analizar la propuesta y tomar una decisión. Las ecuaciones obtenidas para cualquier cantidad de días $t\in[0,T]$ son:

$$
\dot{x_1} = \alpha(x_1,x_2)+\displaystyle\sum_{i=1}^{n}{\delta_{1,i}\cdot\alpha(w_i,x_1)}-\beta_1\cdot x_1
$$
$$
\dot{x_1} = \alpha(x_1,x_2)+\displaystyle\sum_{i=1}^{n}{\gamma_{1,i}\cdot\alpha(w_i,x_1)}-\beta_1\cdot x_1
$$
$$
\dot{x_2} = \alpha(x_2,x_1)+\displaystyle\sum_{i=1}^{n}{\delta_{2,i}\cdot\alpha(w_i,x_2)}-\beta_2\cdot x_2
$$
$$
\dot{x_2} = \alpha(x_2,x_1)+\displaystyle\sum_{i=1}^{n}{\gamma_{2,i}\cdot\alpha(w_i,x_2)}-\beta_2\cdot x_2
$$
$$
\dot{x_1}+\dot{x_2} = \displaystyle\sum_{i=1}^{n}{\epsilon_{i}\cdot\alpha(w_i,\dfrac{x_1+x_2}{2})}-\phi\cdot \dfrac{x_2-x_1}{2}
$$
$$
\dot{x_1}+\dot{x_2} = \displaystyle\sum_{i=1}^{n}{\psi_{i}\cdot\alpha(w_i,\dfrac{x_2-x_1}{2})}-\sigma\cdot \dfrac{x_1+x_2}{2}
$$

donde $\alpha(x_1, x_2)$ es una función que cuantifica la interacción entre ambas poblaciones.

Una posible alternativa sería utilizar solo 2 ecuaciones que involucren utilizar solo a $\dot{x_1}$ y a $\dot{x_2}$, pero esa alternativa queda descartada. Por lo tanto, se le solicita a usted utilizar la información de las 6 ecuaciones diferenciales planteadas para construir las mejores derivadas $\bar{\dot{x_1}}$ y $\bar{\dot{x_2}}$ que aproximen estas ecuaciones con el menor error cuadrático posible.

Como información de utilidad, los científicos sí se pusieron de acuerdo en los valores iniciales para las poblaciones a estudiar: $x_1(0) = x_{1,0}$ y $x_2(0) = x_{2,0}$.


#### 3.1 Explique teóricamente cómo obtener $\bar{\dot{x_1}}$ y $\bar{\dot{x_2}}$, basándose en la información entregada anteriormente. (30 puntos)

Respuesta: 



#### 3.2 Implemente la función $\verb|f_populations|$, donde se implementa de forma explícita el lado derecho del sistema dinámico para $\bar{\dot{x_1}}$ y $\bar{\dot{x_2}}$. Los parámetros que recibe esta función están definidos en la firma respectiva de la función. Notar que el valor del parámetro $n$ se define en función de los vectores $\delta$, $\gamma$, $\epsilon$ y $\psi$. (15 puntos)

In [13]:
def alpha(x1,x2):
    return np.cos(x2*x1)/np.sqrt(np.power(np.cos(x2*x1),2)+np.power(np.cos(0.05),2))

# Codigo solicitado

def f_populations(t, y, w, alpha, beta1, beta2, delta1, delta2, gamma1, gamma2, epsilon, psi, phi, sigma):
    '''
    INPUT:
    t      : double  
             Independent variable t.
    y      : ndarray 
             Input vector that contains x1=y[0] and x2=y[1].
    w      : ndarray 
             Coefficients vector w.
    alpha  : funcion 
             Function already implemented that receives x1 and x2
    beta1  : double
             Coefficient beta_1.
    beta2  : double
             Coefficient beta_2.
    delta1 : ndarray
             Coefficients vector delta_1.
    delta2 : ndarray
             Coefficients vector delta_2.
    gamma1 : ndarray
             Coefficients vector gamma_1.
    gamma2 : ndarray
             Coefficients vector gamma_2.
    epsilon: ndarray
             Coefficients vector epsilon.
    psi    : ndarray
             Coefficients vector psi.
    sigma  : double
             Coefficient sigma.
    phi    : double
             Coefficient phi.
    OUTPUT:
    y_dot  : ndarray
             Output vector that implements the right hand side of the resulting dynamical system that minimizes the squared error.    '''
    # Tu codigo aqui

    return y_dot

#### 3.3 Entregue el valor de $\bar{x_1}(10)$ y de $\bar{x_2}(10)$, considerando que $\bar{x_1}(0)=400$ y $\bar{x_2}(0)=300$ y que utiliza $\verb|solve_ivp|$ de Scipy. El código necesario para obtener el resultado esperado debe considerar los siguientes datos: (5 puntos)

In [14]:
# No modificar

beta1 = 0.25
beta2 = 0.75
phi = 0.341
sigma = 1.41
w = np.array([1.5, 1.625, 1.75, 1.875, 2.])
delta1 = np.array([1.2701176891, 0.28811319, 0.7046913486, 1.6134431034, 1.3446417529])
delta2 = np.array([-0.7036400735, 0.6840636606, -0.10897719, -0.0743175733, 0.2956309214])
gamma1 = np.array([0.1037113712, 1.047076925, 0.5479471621, 0.0876060119, 0.3195815276])
gamma2 = np.array([0.2402455157, 1.0757369327, -0.1477139499, 0.2254087452, -0.6149489323])
epsilon = np.array([-1.8381526674, 0.4706053887, 0.6223940632, -0.5343588147, 1.6342233293])
psi = np.array([1.521645759, 1.2701176891, -0.7036400735, 0.1037113712, 0.2402455157])

# Hacer aqui experimentos



### ¡Ha llegado al final de la tarea 4! No olvide guardar este archivo y enviarlo antes de las 18:00 hrs. ¡Mucho éxito!

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cec3c244-b1da-4213-b94c-4ae359502136' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>